In [1]:
import os
import re
from os.path import exists
import pandas as pd
import csv
from datetime import date

In [2]:
html_location = '/home/justdial/Documents/keywords_rank_input_ws/'

In [3]:
rows = []
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
def cleanhtml(raw_html):
    cleantext =re.sub(CLEANR, '', str(raw_html))
    return cleantext

In [4]:
out_csv = 'insta_urls_ws_new.csv'

csv_exists = exists(out_csv)
if csv_exists:
    print(f"csv exists in path {out_csv},removing now ....")
    os.remove(out_csv)
    
#df = pd.DataFrame(columns=['main_category','leaf_category','ns_price','ns_series','ns_parent_brand','ns_brand','ns_product_UID','ns_model_number','ns_product_group','ns_website_prod.name','ns_displayname','ns_description','ns_features','ns_specifications','ns_extraspec','ns_image_urls','ns_url','ns_active_flag','ns_launch_date','country_of_origin','manufacturer','manufactured_details','importers_detail','importer','product_warranty','ns_video_url','pdf_url','aplus_contents'])
df = pd.DataFrame(columns=['search_string','instagram_url','description','phone1','phone2','phone3','date'])
header_list=df.columns
print(f"creating new CSV...")
df.to_csv(out_csv,sep="|",index=False,header=False)

csv exists in path insta_urls_ws_new.csv,removing now ....
creating new CSV...


In [5]:
count = 0
count2 = 0
ic = 0

for i in os.listdir(html_location):
    if i.endswith(".html"):
        count += 1
        file_path = os.path.join(html_location,i)
#         print(file_path)
        HTMLFile = open(file_path, "r")
        page = HTMLFile.read()
        page = re.sub('\\n','',str(page))
        page = re.sub('<!--(.*?)-->','',str(page))
        page = re.sub('#038;','',str(page))
#         print(page)    
        my_dict = {}
#         search_string = ''
        if 'instagram' in page:
            count2 += 1
#             print(count2)
            # search_string
            try:
                search_string = re.findall('<title>(.*?)</title>',str(page))[0]
            except:
                search_string = ''
            search_string = search_string.replace('www.instagram.com','').replace(' - Google Search','')
#             print('search_string --',search_string)

            today = date.today()
            my_dict['date'] = today.strftime("%d-%m")
            
            my_dict['search_string'] = search_string

            
            # instagram_url
            instagram_url = ''
            insta_urls = []
            ic = 0
            
            if '<div class="A6K0A"' in page:
                print('insta cut found')
#                 ic += 1
                cut_list = page.split('<div class="A6K0A"')
                print('cut_list ==>',len(cut_list))
                print()
                print()
#                 print(cut_list)
                cut_list.pop(0)
                print('cut_list ==>',len(cut_list))
                
                for cut in cut_list:
                    description_l = re.findall('data-snf="nke7rc"(.*?)<\/div><\/div><\/div><\/div><\/div>',str(cut))
                    
                    insta_urls = re.findall('<a jsname="UWckNb" class="zReHs" href="(.*?)"',str(cut))
                    print(insta_urls)
                    if len(insta_urls):
                        my_dict['instagram_url'] = insta_urls[0]
                    else:
                        my_dict['instagram_url'] = ''
                    
                    if len(description_l):
                        description = cleanhtml(description_l[0])
                        description = description.replace('data-sncf="1">','').replace('data-sncf="1,2">','').replace('❤️','').replace('꓄','').replace('➡️','').replace('🕹️','').replace('✨','').replace('☎️','').replace('•','')
#                         print(description)
                        my_dict['description'] = description.strip()
                        
#                         insta_urls = re.findall('<a jsname="UWckNb" class="zReHs" href="(.*?)"',str(description))
#                         if len(insta_urls):
#                             my_dict['instagram_url'] = insta_urls[0]
#                         else:
#                             my_dict['instagram_url'] = ''
                            

                        phones = re.findall(r'\b(?:\d\s*){7,}\b',str(description))
                        print(len(phones))
                        print(phones)
                        
                        if len(phones) == 0:
                            my_dict['phone1'] = ''
                            my_dict['phone2'] = ''
                            my_dict['phone3'] = ''
                            
                        if len(phones) == 1:
                            my_dict['phone1'] = phones[0]
                            my_dict['phone2'] = ''
                            my_dict['phone3'] = ''
                        if len(phones) == 2:
                            my_dict['phone1'] = phones[0]
                            my_dict['phone2'] = phones[1]
                            my_dict['phone3'] = ''
                        if len(phones) == 3:
                            my_dict['phone1'] = phones[0]
                            my_dict['phone2'] = phones[1]
                            my_dict['phone3'] = phones[2]
                            
                        
#                         my_dict['description'] = description
                        
                        df = pd.DataFrame(my_dict, index=[0])
                        list1=list(my_dict.values())
                        print(f"writing to csv .... ")
                        #count+=1
                        with open(out_csv, 'a', newline='') as file:
                            writer = csv.DictWriter(file, fieldnames=header_list,)
                            if file.tell() == 0:
                                writer.writeheader()# Write the data row
                            writer.writerow(my_dict)
                        print('done')

                        
                    
            
            
#             links = re.findall('<a.*?>.*?</a>',str(page))
# #             print(len(links))
#             for link in links:
#                 if 'href="https://www.instagram.com' in link:
#                     insta_links = re.findall('<a.*?href\=\"https\:\/\/www\.instagram\.com(.*?)".*?>',str(link))
# #                     print('insta links --> ',len(insta_links))
                    
# #                     insta_urls = list(set(insta_links))
                    
# #                     for il in insta_links:
# #                         print('il ==>',il)
# #                         if il not in insta_urls:
# #                             il_url = 'https://www.instagram.com' + il
# #                             insta_urls.append(il_url)
                            
                            
#                     for il in insta_links:
#                         full = 'https://www.instagram.com' + il
# #                         print('full url:', full)
#                         if full not in insta_urls:
#                             insta_urls.append(full)

# #             print('insta_urls --',len(insta_urls))
            
#             if len(insta_urls):
#                 for i in insta_urls:
#                     my_dict['instagram_url'] = i
#             else:
#                 my_dict['instagram_url'] = 'url not found'
                
#     else:
#         my_dict['instagram_url'] = 'url not found'
        
        
            

        
#         print(my_func(audio_path))
# print(count2)
#     my_dict['search_string'] = search_string
    
#     df = pd.DataFrame(my_dict, index=[0])
#     list1=list(my_dict.values())
#     print(f"writing to csv .... ")
#     #count+=1
#     with open(out_csv, 'a', newline='') as file:
#         writer = csv.DictWriter(file, fieldnames=header_list,)
#         if file.tell() == 0:
#             writer.writeheader()# Write the data row
#         writer.writerow(my_dict)
#     print('done')


print('ic --',ic)

insta cut found
cut_list ==> 11


cut_list ==> 10
['https://www.instagram.com/explore/locations/318084912328034/motilal-nagar-no1-d-b-more-marg-goregaon-west/']
0
[]
writing to csv .... 
done
['https://www.instagram.com/wholesale999/reel/DCUAncCsCNZ/']
['https://www.instagram.com/explore/locations/429508443/goregaon-west/?locale=en_US%2Cen_US']
1
['91 99672 87488']
writing to csv .... 
done
['https://www.instagram.com/reel/DHlAClZxr9k/']
['https://www.instagram.com/reel/CyiWGEMIX2V/']
['https://www.instagram.com/reel/Cya8m6TxiFj/']
['https://www.instagram.com/explore/locations/248140691/emsquare-studios/']
0
[]
writing to csv .... 
done
['https://housing.com/motilal-nagar-iii-goregaon-west-mumbai-overview-P6l9n8mvtlv4wug7y']
0
[]
writing to csv .... 
done
['https://www.squareyards.com/motilal-nagar-in-mumbai-overview-5470']
0
[]
writing to csv .... 
done
['https://www.instagram.com/reel/DCUAncCsCNZ/']
insta cut found
cut_list ==> 11


cut_list ==> 10
['https://www.cholainsurance.com/ca

insta cut found
cut_list ==> 11


cut_list ==> 10
['https://www.instagram.com/explore/locations/537472365/new-panchwati-colony/']
1
['91 99994 04901']
writing to csv .... 
done
['https://www.instagram.com/explore/locations/1047320865367908/panchwati-ghaziabad/?locale=en']
0
[]
writing to csv .... 
done
['https://www.instagram.com/explore/locations/506786009/panchvati-colony/']
0
[]
writing to csv .... 
done
['https://www.instagram.com/explore/locations/192383061485134/panchwati-extension/']
0
[]
writing to csv .... 
done
['https://www.squareyards.com/panchvati-colony-in-ghaziabad-overview-10717']
0
[]
writing to csv .... 
done
['https://www.instagram.com/explore/locations/1047320865367908/panchwati-ghaziabad/?locale=English']
0
[]
writing to csv .... 
done
['https://www.justdial.com/Ghaziabad/Wedding-Gown-Retailers-in-Chhaprola/nct-10534719']
0
[]
writing to csv .... 
done
['https://www.instagram.com/explore/locations/381345232273342/panchwati/']
0
[]
writing to csv .... 
done
['https:

insta cut found
cut_list ==> 11


cut_list ==> 10
['https://news.abplive.com/pincode/telangana/k-v-rangareddy/nandiwanaparthy-pincode-501509.html']
0
[]
writing to csv .... 
done
['https://www.goodreturns.in/nandiwanaparthy-b-o-pin-code-501509-d502.html']
0
[]
writing to csv .... 
done
['https://www.aarral.in/sale/online/pincode/501509?srsltid=AfmBOorcmsU6sE04_UvxSeCaK_vepvBwnONzaIQ8QfTQtC5V_mOm43ma']
0
[]
writing to csv .... 
done
['https://news.abplive.com/pincode/telangana/k-v-rangareddy/tatiparthy-pincode-501509.html']
0
[]
writing to csv .... 
done
['https://pincodes.nskmultiservices.in/pincodes/Telangana/rangareddy/nallavelli']
0
[]
writing to csv .... 
done
['https://www.abplive.com/pincode/telangana/k-v-rangareddy/kothapalli-pincode-501509.html']
0
[]
writing to csv .... 
done
['https://ndtv.in/tools/pincodes-india/telangana/kvrangareddy']
0
[]
writing to csv .... 
done
['https://www.aarral.in/sale/online/place/501509-manthangoud-gowrelly-ranga-reddy-telangana-india?srsltid=Afm

insta cut found
cut_list ==> 11


cut_list ==> 10
['https://www.falconebiz.com/company/421306.html']
0
[]
writing to csv .... 
done
['https://www.creditmantri.com/ifsc-code-bank-of-baroda-maharashtra-kalyan-tisgaon-naka-kalyan-e-branch/']
2
['1800223344', '400012104']
writing to csv .... 
done
['https://branch.axisbank.com/location/maharashtra/thane/katemanivali']
1
['18605005555']
writing to csv .... 
done
['https://mygate.com/kalyan/shree-rajshree-dhara-co-op-hsg-soc-ltd-4dcb.html']
0
[]
writing to csv .... 
done
['https://ifsc.bankifsccode.com/BARB0TISGAO']
0
[]
writing to csv .... 
done
['https://www.magicbricks.com/pin-code/wadi-number-421306']
0
[]
writing to csv .... 
done
['https://branches.muthootfincorp.com/muthoot-fincorp-gold-loan-katemanivli-loan-agency-katemanivali-thane-117513/Home']
1
['09289287802']
writing to csv .... 
done
['https://villageinfo.in/maharashtra/thane/kalyan/pisawali.html']
0
[]
writing to csv .... 
done
['https://www.instagram.com/kalyan_narayanan/']
0

insta cut found
cut_list ==> 11


cut_list ==> 10
['https://www.instagram.com/tiepune/?hl=en']
0
[]
writing to csv .... 
done
['https://www.icicibank.com/branch/maharashtra/pune/icic0000424']
0
[]
writing to csv .... 
done
['https://www.justdial.com/Pune/Gurukul-School-Behind-E-Square-Multiplex-Near-Vijay-Housing-Society-Shivaji-Nagar-Model-Colony/020P5086091_BZDET']
0
[]
writing to csv .... 
done
['https://news.abplive.com/pincode/maharashtra/pune/model-colony-pincode-411016.html']
0
[]
writing to csv .... 
done
['https://www.nobroker.in/pg-for-ladies-near-blue_bird_society_pune']
0
[]
writing to csv .... 
done
['https://stores.charlottetilbury.com/in/maharashtra/pune/nykaa-on-trend-g-06-senapati-bapat-rd-next-to-jw-marriott-hotel-laxmi-society-model-colony-shivajinagar-pune-maharashtra-411016']
0
[]
writing to csv .... 
done
['https://www.instagram.com/rohanbuilders/?hl=en']
0
[]
writing to csv .... 
done
['https://www.instagram.com/afpune/?hl=en']
1
['91 83790 90666']
writing to csv

...

insta cut found
cut_list ==> 12


cut_list ==> 11
['https://news.abplive.com/pincode/maharashtra/thane/manivali-pincode-421102.html']
0
[]
writing to csv .... 
done
['https://news.abplive.com/pincode/maharashtra/thane/mohone-pincode-421102.html#:~:text=Mohone%20Pin%20Code%20is%20421102,Mohone%20comes%20under%20Thane%20district.', 'https://www.indiatvnews.com/pincode/maharashtra/thane/atali#:~:text=Atali%20Pin%20Code%20is%20421102,district%20Thane%2C%20Maharashtra%2C%20INDIA.', 'https://news.abplive.com/pincode/maharashtra/thane/shahad-pincode-421103.html#:~:text=Shahad%20Pin%20Code%20is%20421103,Shahad%20comes%20under%20Thane%20district.']
['https://www.abplive.com/pincode/maharashtra/thane/manivali-pincode-421102.html']
0
[]
writing to csv .... 
done
['https://www.ndtv.com/tools/pincodes/maharashtra/thane/mohone-so']
0
[]
writing to csv .... 
done
['https://www.justdial.com/Thane/Somus-Momos-Near-Sadanand-Tower-Ambivali/022PXX22-XX22-220311130243-M9L9_BZDET']
0
[]
writing to csv .... 

insta cut found
cut_list ==> 11


cut_list ==> 10
['https://www.instagram.com/usaranivillage/?hl=en']
0
[]
writing to csv .... 
done
['https://www.instagram.com/tadakarestrorant/']
0
[]
writing to csv .... 
done
['https://www.instagram.com/apratimorganicfarm/']
0
[]
writing to csv .... 
done
['https://www.instagram.com/omtexicwc/reel/C94bgIAoapH/']
0
[]
writing to csv .... 
done
['https://www.instagram.com/patilfarmhouse/reels/']
0
[]
writing to csv .... 
done
['https://www.instagram.com/zinal_builders/']
0
[]
writing to csv .... 
done
['https://housing.com/pin-code/dhekale-palghar-pin-code-401102']
0
[]
writing to csv .... 
done
['https://news.abplive.com/pincode/maharashtra/thane/tembhikhodave-pincode-401102.html']
0
[]
writing to csv .... 
done
['https://www.ndtv.com/tools/pincodes/maharashtra/thane/umbarpada-so']
0
[]
writing to csv .... 
done
['https://www.indiatvnews.com/pincode/maharashtra/thane/pargaon']
0
[]
writing to csv .... 
done
insta cut found
cut_list ==> 11


cut_list 

insta cut found
cut_list ==> 12


cut_list ==> 11
[]
0
[]
writing to csv .... 
done
[]
1
['91 79999 98212']
writing to csv .... 
done
[]
0
[]
writing to csv .... 
done
[]
0
[]
writing to csv .... 
done
[]
0
[]
writing to csv .... 
done
[]
0
[]
writing to csv .... 
done
[]
0
[]
writing to csv .... 
done
[]
0
[]
writing to csv .... 
done
[]
0
[]
writing to csv .... 
done
[]
0
[]
writing to csv .... 
done
[]
insta cut found
cut_list ==> 2


cut_list ==> 1
[]
insta cut found
cut_list ==> 11


cut_list ==> 10
['https://www.instagram.com/oyeyaadi/reel/DE4tcj9PcX0/']
['https://theimperialindia.com/wellness/salon-imperial/']
0
[]
writing to csv .... 
done
['https://spvdelhi.org/']
0
[]
writing to csv .... 
done
['https://nzpnewdelhi.gov.in/']
0
[]
writing to csv .... 
done
['https://cafe.hardrock.com/new-delhi/']
0
[]
writing to csv .... 
done
['https://www.international.gc.ca/country-pays/india-inde/new_delhi.aspx?lang=eng']
0
[]
writing to csv .... 
done
['https://www.kln.gov.my/web/ind_new-

insta cut found
cut_list ==> 9


cut_list ==> 8
['https://www.instagram.com/ogrolls.india/']
0
[]
writing to csv .... 
done
['https://www.instagram.com/loudkanpur/?hl=en']
2
['88969 22129', '88582 79787']
writing to csv .... 
done
['https://www.instagram.com/sevendaysgym_official/']
0
[]
writing to csv .... 
done
['https://www.instagram.com/shyam_nagar01/?hl=en']
0
[]
writing to csv .... 
done
['https://www.instagram.com/jogeshwari60official/?api=%E9%A9%AC%E6%9D%A5%E8%A5%BF%E4%BA%9A%E5%B3%87%E6%A0%AA%E5%B7%B4%E8%BE%96%E7%BA%A6%E7%82%AEWhatsApp%EF%BC%9A601167898268%E5%85%A8%E5%A5%97%E6%9C%8D%E5%8A%A1.ogrm&amp;hl=zh-cn']
0
[]
writing to csv .... 
done
['https://www.instagram.com/explore/locations/226407938128283/shyam-nagar-talav/']
0
[]
writing to csv .... 
done
['https://www.instagram.com/sanjivani.dental.clinic/']
1
['91 7400301463']
writing to csv .... 
done
[]
insta cut found
cut_list ==> 2


cut_list ==> 1
[]
insta cut found
cut_list ==> 11


cut_list ==> 10
['https://www.instagram

insta cut found
cut_list ==> 11


cut_list ==> 10
['https://www.99acres.com/chinchpada-gaon-mumbai-beyond-thane-overview-piffid']
0
[]
writing to csv .... 
done
['https://www.propscience.com/location_details_site_address/UDUxNzAwMDI2MDY3/Kalyan-Dombivli/Kalyan/Kalyan-East/Dwarli-Gaon/Mali-Pinnacle']
0
[]
writing to csv .... 
done
['https://www.icicibank.com/atm/maharashtra/kalyan/kalyan/a6486']
1
['18001080']
writing to csv .... 
done
['https://www.michelin.in/motorbike/dealer-locator/thane/ykybblo-super-wheel-care']
0
[]
writing to csv .... 
done
['https://www.instagram.com/reel/C4vFh5JiyLq/']
['https://branches.muthootfincorp.com/muthoot-fincorp-gold-loan-katemanivli-loan-agency-katemanivali-thane-117513/Contact-Us']
0
[]
writing to csv .... 
done
['https://restaurantswowchicken.wowmomo.com/wow-chicken-metro-junction-mall-kalyan-chicken-restaurants-kalyan-west-kalyan-286859/Contact-Us']
2
['919147318040', '919836198361']
writing to csv .... 
done
['https://razorpay.com/ifsc-code/bank

In [6]:
print(date)

NameError: name 'date' is not defined